In [1]:
import numpy as np
import os
import pathlib
import pandas as pd
import pandasql as ps

In [2]:
#read file
cwd = os.getcwd()
filepath = list(pathlib.Path(cwd).rglob('Senior_Data_Analyst_Data_Challenge.csv'))

df = pd.read_csv(filepath[0])

#to proper datetime format
df['observed_at'] = pd.to_datetime(df['observed_at'])

In [141]:
#pandasql enables sqlite syntax on pandas dataframe
q1 = """select machine, application_name, COUNT(*) as count, round(100 * COUNT(*) / CAST(SUM(count(*)) over (partition by machine) as float),2) as pct_user_total from df group by 1, 2"""

result = ps.sqldf(q1, locals())

display(result)

,machine,application_name,count,pct_user_total
0,PC-WinLab-BP,Application Frame Host,9,0.03
1,PC-WinLab-BP,Automate,5503,21.18
2,PC-WinLab-BP,Chrome,585,2.25
3,PC-WinLab-BP,Csrss,1,0.00
4,PC-WinLab-BP,Dllhost,12,0.05
5,PC-WinLab-BP,Dwm,3,0.01
6,PC-WinLab-BP,Excel,1481,5.70
7,PC-WinLab-BP,Explorer,1005,3.87
8,PC-WinLab-BP,Iexplore,8,0.03
9,PC-WinLab-BP,Neo2,1,0.00


In [168]:
#here are the 3 top consecutive title runs of length 5 
q3 = """select t1, t2, t3, t4, t5, count(*) from (select machine, title as t1, observed_at, lead(title,1) OVER (partition by machine ORDER by 1,3) as t2, lead(title,2) OVER (partition by machine ORDER by 1,3) as t3, lead(title,3) OVER (partition by machine ORDER by 1,3) as t4, lead(title,4) OVER (partition by machine ORDER by 1,3) as t5 from df) where t1!=t2 and t2!=t3 and t3!=t4 and t4!=t5 group by 1,2,3,4,5 order by 6 desc limit 3"""

result = ps.sqldf(q3, locals())

display(result)

,t1,t2,t3,t4,t5,count(*)
0,TypeAheadPopup,Vendor Line Item Display,Define Filter Criteria,Filter criteria,Vendor Line Item Display,69
1,Display Document: Initial Screen,Display Document: Data Entry View,Post Outgoing Payments: Header Data,Post Outgoing Payments Enter selection criteria,Post Outgoing Payments Process open items,65
2,Post Outgoing Payments Process open items,Post Outgoing Payments: Header Data,Display Document: Initial Screen,Display Document: Data Entry View,TypeAheadPopup,65


In [154]:
#here are the 5 top most common application switches 
q2 = """select app_name,next_app, count(*) from (select machine, application_name as app_name, observed_at, lag(application_name,-1) OVER (partition by machine ORDER by 1,3) as next_app from df) where app_name!=next_app group by 1,2 order by 3 desc limit 5"""

result = ps.sqldf(q2, locals())

display(result)

,app_name,next_app,count(*)
0,Automate,Saplogon,344
1,Saplogon,Automate,331
2,Explorer,Excel,135
3,Automate,Explorer,133
4,Explorer,Automate,124


In [129]:
#sum of active time per user
active_time = df.sort_values('observed_at')[['machine','observed_at']]
active_time['observed_at_lag'] = active_time['observed_at'].shift(1)
active_time['time_bw_events'] = active_time['observed_at'] - active_time['observed_at_lag']
display(active_time['time_bw_events'][active_time['machine']=='PC-WinLab-BP'].quantile(.99))
display(active_time['time_bw_events'][active_time['machine']=='PC-WinLab-RS'].quantile(.99))

Timedelta('0 days 00:01:18.429319')

Timedelta('0 days 00:15:50.632559')

In [130]:
#You can immediately see that the time deltas are heavily skewed towards small intervals. If we filter out those outliers, we should have a pretty good sense of total active time per user.
display(active_time['time_bw_events'][active_time['machine']=='PC-WinLab-BP'].max())
display(active_time['time_bw_events'][active_time['machine']=='PC-WinLab-RS'].max())

Timedelta('2 days 11:43:49.936000')

Timedelta('0 days 04:10:43.817000')

In [140]:
#let's sum up time deltas using the 99th quantile values above
BP_qtl = active_time['time_bw_events'][active_time['machine']=='PC-WinLab-BP'].quantile(.99)
RS_qtl = active_time['time_bw_events'][active_time['machine']=='PC-WinLab-RS'].quantile(.99)
display(active_time['time_bw_events'][(active_time['machine']=='PC-WinLab-BP') & (active_time['time_bw_events']<BP_qtl)].sum())
display(active_time['time_bw_events'][(active_time['machine']=='PC-WinLab-RS') & (active_time['time_bw_events']<RS_qtl)].sum())

Timedelta('1 days 23:11:27.913000')

Timedelta('0 days 03:54:14.446000')